In [ ]:
실습N #1: 엔비디아의 주가 데이터를 가져와서 분석하는 어플리케이션을 작성하시오.
실습N #2: 사용자가 입력한 지역과 질문에 맞는 지역 음식점을 검색해서 추천하는 어플리케이션을 작성하시오.


In [ ]:
!pip install yfinance

In [ ]:
# 실습N #1: 엔비디아의 주가 데이터를 가져와서 분석하는 어플리케이션을 작성하시오.
import yfinance as yf
import openai

nvda = yf.Ticker("NVDA")

nvda_data = nvda.history(start="2022-01-01", end="2024-12-31")

monthly_data = nvda_data.resample('M').mean()

print(monthly_data.head(50))

monthly_data = monthly_data.head(50).to_string()

# OpenAI API 요청 준비
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "다음 주식 데이터를 상세하게 분석해주세요."},  
        {"role": "user", "content": monthly_data},
        {"role": "assistant","content": "엔비디아의 주식을 보고 투자를 해야할지 말아야할지 분석해줘"}
    ],
    temperature=0.5,
    max_tokens=1024,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(response.choices[0].message.content)



In [ ]:
import os
import json
import openai
from openai import OpenAI
from pykrx import stock

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

import json

# input으로 start_date,end_date, ticker 받기 ticker는 gpt가 찾아서 적용

start_date = input("시작 날짜를 입력하세요 ex)2022-07-20 : ")
end_date = input("시작 날짜를 입력하세요 ex)2022-07-20 : ")
input_ticker = input("찾으려는 종목을 입력하세요 : ")

res = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "다음 종목의 ticker를 찾아서 숫자로만 출력하세요."},  
        {"role": "user", "content": input_ticker}
    ],
    temperature=0.5,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# 분석 결과 출력
ticker = res.choices[0].message.content



def get_market_ohlcv(start_date, end_date, ticker):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)


tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_market_ohlcv",
            "description": "특정 종목에 대해 정해진 기간의 주가 데이터를 돌려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 시작 날짜",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 종료 날짜",
                    },
                    "ticker": {
                        "type": "string",
                        "description": "6자리 숫자 형식의 종목 티커코드",
                    },
                },
                "required": ["start_date", "end_date", "ticker"],
            },
        }
    }
]


messages= [
  {
    "role": "user",
    "content": f"{ticker}의 {start_date}부터 {end_date}까지의 주가 데이터를 분석해줘."
  }
]

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=messages,
  temperature=0,
  max_tokens=1024,
  tools=tools,
  tool_choice="auto" if tools is not None else None,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)


available_functions = {
    "get_market_ohlcv": get_market_ohlcv,
}

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
        ticker=function_args['ticker']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)
